In [1]:
import pandas as pd
from datetime import datetime, timezone
from src.processing.transform_guardian import clean_html
import re

In [2]:
import pandas as pd
from datetime import datetime, timezone
import json
from src.processing.transform_guardian import clean_html  # nếu bạn có hàm này

# ---- Đọc JSON file ----
data_path = 'data/articles_20251007_072317.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# ---- Chuyển JSON → DataFrame ----
df = pd.json_normalize(data, sep="_")

# ---- Giữ lại các cột cần thiết ----
cols_to_keep = [
    "id", "type", "sectionId", "sectionName",
    "webPublicationDate", "webTitle",
    "fields_headline", "fields_trailText",
    "fields_byline", "fields_wordcount",
    "fields_publication", "fields_thumbnail",
    "fields_body",    # 🆕 Thêm phần nội dung chính
    "pillarName", "webUrl"
]
df = df[cols_to_keep]

# ---- Chuẩn hóa kiểu dữ liệu ----
df["webPublicationDate"] = pd.to_datetime(df["webPublicationDate"], errors="coerce")
df["fields_wordcount"] = pd.to_numeric(df["fields_wordcount"], errors="coerce")

# ---- 🧹 Làm sạch dữ liệu ----
df["fields_trailText"] = df["fields_trailText"].apply(lambda x: clean_html(x) if isinstance(x, str) else x)
df["fields_headline"] = df["fields_headline"].apply(lambda x: clean_html(x) if isinstance(x, str) else x)
df["fields_body"] = df["fields_body"].apply(lambda x: clean_html(x) if isinstance(x, str) else x)
df["fields_body"] = df["fields_body"].fillna("").str.strip()
df["fields_byline"] = df["fields_byline"].fillna("Unknown").str.title()
df["sectionName"] = df["sectionName"].fillna("Unknown").str.title()
df["webTitle"] = df["webTitle"].fillna("").str.strip()
df["fields_publication"] = df["fields_publication"].fillna("The Guardian")

# ---- 🚀 Feature Engineering ----
df["title_length"] = df["webTitle"].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df["headline_length"] = df["fields_headline"].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df["days_since_publication"] = (
    pd.Timestamp.now(tz=timezone.utc) - df["webPublicationDate"]
).dt.days
df["has_thumbnail"] = df["fields_thumbnail"].notnull().astype(int)

# ---- 📋 Metadata Tracking ----
df["ingested_at"] = datetime.now(timezone.utc)
df["source_system"] = "guardian_api"
df["raw_s3_key"] = data_path   # vì đang test local, thay vì S3 key
df["processed_by"] = "transform_guardian_v2"

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   id                      10 non-null     object             
 1   type                    10 non-null     object             
 2   sectionId               10 non-null     object             
 3   sectionName             10 non-null     object             
 4   webPublicationDate      10 non-null     datetime64[ns, UTC]
 5   webTitle                10 non-null     object             
 6   fields_headline         10 non-null     object             
 7   fields_trailText        10 non-null     object             
 8   fields_byline           10 non-null     object             
 9   fields_wordcount        10 non-null     int64              
 10  fields_publication      10 non-null     object             
 11  fields_thumbnail        10 non-null     object  

In [3]:
df

,id,type,sectionId,sectionName,webPublicationDate,webTitle,fields_headline,fields_trailText,fields_byline,fields_wordcount,...,pillarName,webUrl,title_length,headline_length,days_since_publication,has_thumbnail,ingested_at,source_system,raw_s3_key,processed_by
0,business/live/2025/oct/07/uk-house-price-growt...,liveblog,business,Business,2025-10-07 07:23:10+00:00,UK house price growth slowest since April 2024...,UK house price growth slowest since April 2024...,Rolling coverage of the latest economic and fi...,Graeme Wearden,1423,...,News,https://www.theguardian.com/business/live/2025...,21,21,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
1,world/live/2025/oct/07/gaza-ceasefire-peace-ta...,liveblog,world,World News,2025-10-07 07:21:22+00:00,Gaza ceasefire talks live: negotiations to con...,Gaza ceasefire talks live: negotiations to con...,The first day of Gaza talks between Hamas and ...,Yohannes Lowe,679,...,News,https://www.theguardian.com/world/live/2025/oc...,16,16,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
2,politics/live/2025/oct/07/conservative-tory-co...,liveblog,politics,Politics,2025-10-07 07:21:08+00:00,Badenoch defends Jenrick over comments that he...,Badenoch defends Jenrick over comments that he...,Conservative leader suggests remarks in Guardi...,Andrew Sparrow,1939,...,News,https://www.theguardian.com/politics/live/2025...,19,19,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
3,australia-news/live/2025/oct/07/australia-poli...,liveblog,australia-news,Australia News,2025-10-07 07:18:00+00:00,New footprint unrelated to search for Gus Lamo...,New footprint unrelated to search for Gus Lamo...,This blog is now closed,Adeshola Ore And Krishani Dhanji (Earlier),11756,...,News,https://www.theguardian.com/australia-news/liv...,15,15,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
4,money/2025/oct/07/uk-house-prices-buyers-budge...,article,money,Money,2025-10-07 07:17:10+00:00,UK house price growth slows as buyers show cau...,UK house price growth slows as buyers show cau...,"Cost of a typical home fell £794 to £298,184 i...",Julia Kollewe,465,...,Lifestyle,https://www.theguardian.com/money/2025/oct/07/...,11,11,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
5,football/2025/oct/07/revamped-womens-champions...,article,football,Football,2025-10-07 07:00:35+00:00,Revamped Women’s Champions League: what has ch...,Revamped Women’s Champions League: what has ch...,As the new league phase kicks off we look at w...,Suzanne Wrack,837,...,Sport,https://www.theguardian.com/football/2025/oct/...,12,12,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
6,commentisfree/2025/oct/07/tory-party-britain-k...,article,commentisfree,Opinion,2025-10-07 07:00:35+00:00,I didn’t think I’d ever say this: let’s hope t...,I didn’t think I’d ever say this: let’s hope t...,Britain badly needs a grownup party of the cen...,Polly Toynbee,982,...,Opinion,https://www.theguardian.com/commentisfree/2025...,18,15,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
7,politics/2025/oct/07/keir-starmer-calls-studen...,article,politics,Politics,2025-10-07 06:52:53+00:00,Keir Starmer calls student protests on 7 Octob...,Keir Starmer calls student protests on 7 Octob...,Prime minister urges students not to protest o...,Jamie Grierson,606,...,News,https://www.theguardian.com/politics/2025/oct/...,10,10,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2
8,books/2025/oct/07/the-devil-book-by-asta-olivi...,article,books,Books,2025-10-07 06:00:35+00:00,The Devil Book by Asta Olivia Nordenhof review...,The D

In [4]:
df.head(1)


,id,type,sectionId,sectionName,webPublicationDate,webTitle,fields_headline,fields_trailText,fields_byline,fields_wordcount,...,pillarName,webUrl,title_length,headline_length,days_since_publication,has_thumbnail,ingested_at,source_system,raw_s3_key,processed_by
0,business/live/2025/oct/07/uk-house-price-growt...,liveblog,business,Business,2025-10-07 07:23:10+00:00,UK house price growth slowest since April 2024...,UK house price growth slowest since April 2024...,Rolling coverage of the latest economic and fi...,Graeme Wearden,1423,...,News,https://www.theguardian.com/business/live/2025...,21,21,7,1,2025-10-15 06:42:21.130856+00:00,guardian_api,data/articles_20251007_072317.json,transform_guardian_v2


business/live/2025/oct/07/uk-house-price-growth-slowest-april-2024-trump-truck-imports-25-tariff-1-november-stock-markets-business-live-news

In [5]:
def extract_from_id(article_id):
    if not isinstance(article_id, str):
        return pd.Series({
            "article_section": None,
            "article_type": None,
            "article_year": None,
            "article_month": None,
            "article_day": None,
            "slug_keywords": [],
            "slug_length": None,
            "is_live_blog": None,
            "topic_country": None
        })

    parts = article_id.split('/')
    # Tìm vị trí của năm đầu tiên (4 chữ số)
    year_idx = next((i for i, p in enumerate(parts) if re.fullmatch(r"\d{4}", p)), None)
    if year_idx is None:
        return pd.Series(...)

    section = parts[0] if len(parts) > 0 else None
    article_type = parts[1] if year_idx > 1 else None  # chỉ có type nếu có phần giữa section và year
    year = int(parts[year_idx])
    month = parts[year_idx + 1] if len(parts) > year_idx + 1 else None
    day = int(parts[year_idx + 2]) if len(parts) > year_idx + 2 and parts[year_idx + 2].isdigit() else None

    slug_parts = parts[year_idx + 3:]
    slug = "-".join(slug_parts)

    slug_keywords = re.findall(r"[a-zA-Z]+", slug)
    slug_length = len(slug_keywords)
    is_live_blog = article_type and "live" in article_type.lower()
    known_countries = {"uk", "us", "china", "india", "australia", "eu"}
    topic_country = next((w for w in slug_keywords if w.lower() in known_countries), None)

    return pd.Series({
        "article_section": section,
        "article_type": article_type,
        "article_year": year,
        "article_month": month,
        "article_day": day,
        "slug_keywords": slug_keywords,
        "slug_length": slug_length,
        "is_live_blog": bool(is_live_blog),
        "topic_country": topic_country
    })


# Gắn các cột mới vào DataFrame
df = df.join(df["id"].apply(extract_from_id))

In [6]:
df.head()

,id,type,sectionId,sectionName,webPublicationDate,webTitle,fields_headline,fields_trailText,fields_byline,fields_wordcount,...,processed_by,article_section,article_type,article_year,article_month,article_day,slug_keywords,slug_length,is_live_blog,topic_country
0,business/live/2025/oct/07/uk-house-price-growt...,liveblog,business,Business,2025-10-07 07:23:10+00:00,UK house price growth slowest since April 2024...,UK house price growth slowest since April 2024...,Rolling coverage of the latest economic and fi...,Graeme Wearden,1423,...,transform_guardian_v2,business,live,2025,oct,7,"[uk, house, price, growth, slowest, april, tru...",16,True,uk
1,world/live/2025/oct/07/gaza-ceasefire-peace-ta...,liveblog,world,World News,2025-10-07 07:21:22+00:00,Gaza ceasefire talks live: negotiations to con...,Gaza ceasefire talks live: negotiations to con...,The first day of Gaza talks between Hamas and ...,Yohannes Lowe,679,...,transform_guardian_v2,world,live,2025,oct,7,"[gaza, ceasefire, peace, talks, israel, hamas,...",15,True,None
2,politics/live/2025/oct/07/conservative-tory-co...,liveblog,politics,Politics,2025-10-07 07:21:08+00:00,Badenoch defends Jenrick over comments that he...,Badenoch defends Jenrick over comments that he...,Conservative leader suggests remarks in Guardi...,Andrew Sparrow,1939,...,transform_guardian_v2,politics,live,2025,oct,7,"[conservative, tory, conference, kemi, badenoc...",15,True,uk
3,australia-news/live/2025/oct/07/australia-poli...,liveblog,australia-news,Australia News,2025-10-07 07:18:00+00:00,New footprint unrelated to search for Gus Lamo...,New footprint unrelated to search for Gus Lamo...,This blog is now closed,Adeshola Ore And Krishani Dhanji (Earlier),11756,...,transform_guardian_v2,australia-news,live,2025,oct,7,"[australia, politics, live, october, anniversa...",18,True,australia
4,money/2025/oct/07/uk-house-prices-buyers-budge...,article,money,Money,2025-10-07 07:17:10+00:00,UK house price growth slows as buyers show cau...,UK house price growth slows as buyers show cau...,"Cost of a typical home fell £794 to £298,184 i...",Julia Kollewe,465,...,transform_guardian_v2,money,None,2025,oct,7,"[uk, house, prices, buyers, budget, halifax]",6,False,uk


In [7]:
print(df.columns.tolist())

['id', 'type', 'sectionId', 'sectionName', 'webPublicationDate', 'webTitle', 'fields_headline', 'fields_trailText', 'fields_byline', 'fields_wordcount', 'fields_publication', 'fields_thumbnail', 'fields_body', 'pillarName', 'webUrl', 'title_length', 'headline_length', 'days_since_publication', 'has_thumbnail', 'ingested_at', 'source_system', 'raw_s3_key', 'processed_by', 'article_section', 'article_type', 'article_year', 'article_month', 'article_day', 'slug_keywords', 'slug_length', 'is_live_blog', 'topic_country']


In [8]:
df.iloc[0]

id                        business/live/2025/oct/07/uk-house-price-growt...
type                                                               liveblog
sectionId                                                          business
sectionName                                                        Business
webPublicationDate                                2025-10-07 07:23:10+00:00
webTitle                  UK house price growth slowest since April 2024...
fields_headline           UK house price growth slowest since April 2024...
fields_trailText          Rolling coverage of the latest economic and fi...
fields_byline                                                Graeme Wearden
fields_wordcount                                                       1423
fields_publication                                          theguardian.com
fields_thumbnail          https://media.guim.co.uk/b1efc890c0b15b7be7f1f...
fields_body               8.23am BST    JLR announces some manufacturing...
pillarName  

In [9]:
df[["id", "sectionId", "article_section"]].head(10)

,id,sectionId,article_section
0,business/live/2025/oct/07/uk-house-price-growt...,business,business
1,world/live/2025/oct/07/gaza-ceasefire-peace-ta...,world,world
2,politics/live/2025/oct/07/conservative-tory-co...,politics,politics
3,australia-news/live/2025/oct/07/australia-poli...,australia-news,australia-news
4,money/2025/oct/07/uk-house-prices-buyers-budge...,money,money
5,football/2025/oct/07/revamped-womens-champions...,football,football
6,commentisfree/2025/oct/07/tory-party-britain-k...,commentisfree,commentisfree
7,politics/2025/oct/07/keir-starmer-calls-studen...,politics,politics
8,books/2025/oct/07/the-devil-book-by-asta-olivi...,books,books
9,technology/2025/oct/07/airpods-pro-3-review-ba...,technology,technology


In [10]:
cols_to_drop = ["sectionId", "article_section"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


In [11]:
print(df.columns.tolist())


['id', 'type', 'sectionName', 'webPublicationDate', 'webTitle', 'fields_headline', 'fields_trailText', 'fields_byline', 'fields_wordcount', 'fields_publication', 'fields_thumbnail', 'fields_body', 'pillarName', 'webUrl', 'title_length', 'headline_length', 'days_since_publication', 'has_thumbnail', 'ingested_at', 'source_system', 'raw_s3_key', 'processed_by', 'article_type', 'article_year', 'article_month', 'article_day', 'slug_keywords', 'slug_length', 'is_live_blog', 'topic_country']


In [12]:
df = df.rename(columns={"id": "article_id"})

print(df.columns.tolist())


['article_id', 'type', 'sectionName', 'webPublicationDate', 'webTitle', 'fields_headline', 'fields_trailText', 'fields_byline', 'fields_wordcount', 'fields_publication', 'fields_thumbnail', 'fields_body', 'pillarName', 'webUrl', 'title_length', 'headline_length', 'days_since_publication', 'has_thumbnail', 'ingested_at', 'source_system', 'raw_s3_key', 'processed_by', 'article_type', 'article_year', 'article_month', 'article_day', 'slug_keywords', 'slug_length', 'is_live_blog', 'topic_country']


In [13]:
data_path1 = 'data/guardian_articles_20251008_030002.parquet'

df_p = pd.read_parquet(data_path1)

df_p


,article_id,type,sectionName,webPublicationDate,webTitle,fields_headline,fields_trailText,fields_byline,fields_wordcount,fields_publication,...,raw_s3_key,processed_by,article_type,article_year,article_month,article_day,slug_keywords,slug_length,is_live_blog,topic_country
0,australia-news/live/2025/oct/08/australia-poli...,liveblog,Australia News,2025-10-08 02:58:31+00:00,Australia politics live: Price says leaks risk...,Australia politics live: Price says leaks risk...,Follow live,Krishani Dhanji,7812,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,live,2025,oct,8,"[australia, politics, live, gaza, rally, parli...",15,True,australia
1,us-news/live/2025/oct/07/donald-trump-national...,liveblog,Us News,2025-10-08 02:06:57+00:00,Oregon’s governor says she told Kristi Noem ‘t...,Oregon’s governor says she told Kristi Noem ‘t...,"Tina Kotek, Oregon’s governor, says in stateme...","Robert Mackey (Now); Shrai Popat, Lucy Campbel...",10078,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,live,2025,oct,7,"[donald, trump, national, guard, pam, bondi, m...",15,True,us
2,music/2025/oct/08/dolly-parton-health-problems...,article,Music,2025-10-08 01:21:12+00:00,"Dolly Parton ‘going to be just fine’, singer’s...","Dolly Parton ‘going to be just fine’, singer’s...",Freida Parton says she has been ‘up all night ...,Sian Cain,426,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[dolly, parton, health, problems, sister, pray...",11,False,None
3,australia-news/2025/oct/08/alleged-kill-team-c...,article,Australia News,2025-10-08 01:07:16+00:00,Alleged ‘kill team’ charged after arrest en ro...,Alleged ‘kill team’ charged after arrest en ro...,Police allege they identified the trio ‘mobili...,Nick Visser,501,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[alleged, kill, team, charged, after, arrest, ...",13,False,None
4,world/2025/oct/08/new-zealand-oceans-warming-3...,article,World News,2025-10-08 00:57:22+00:00,New Zealand oceans warming 34% faster than glo...,New Zealand oceans warming 34% faster than glo...,NZ$180bn worth of housing and $26bn of infrast...,Eva Corlett,631,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[new, zealand, oceans, warming, per, cent, fas...",10,False,None
5,australia-news/2025/oct/08/victoria-free-weeke...,article,Australia News,2025-10-08 00:54:29+00:00,Victorians to get free weekend public transpor...,Victorians to get free weekend public transpor...,"Fares waived for weekend trams, trains and bus...",Benita Kolovos Victorian State Correspondent,528,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[victoria, free, weekend, public, transport, s...",6,False,None
6,us-news/2025/oct/08/trump-administration-news-...,article,Us News,2025-10-08 00:40:04+00:00,Trump news at a glance: Texas national guard a...,Trump news at a glance: Texas national guard a...,Military presence comes after US district judg...,Guardian Staff,834,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[trump, administration, news, updates, today]",5,False,None
7,world/2025/oct/08/ukraine-war-briefing-shadow-...,article,World News,2025-10-08 00:33:23+00:00,Ukraine war briefing: Shadow fleet is launchpa...,Ukraine war briefing: Shadow fleet is launchpa...,Tankers being used for ‘reconnaissance and sab...,Warren Murray With Guardian Writers And Agencies,650,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[ukraine, war, briefing, shadow, fleet, is, la...",13,False,None
8,us-news/2025/oct/07/zohran-mamdani-statement-o...,article,Us News,2025-10-08 00:27:48+00:00,Mamdani attends Israelis for Peace vigil after...,Mamdani attends Israelis for Peace vigil 

In [14]:
df_p.head(3)

,article_id,type,sectionName,webPublicationDate,webTitle,fields_headline,fields_trailText,fields_byline,fields_wordcount,fields_publication,...,raw_s3_key,processed_by,article_type,article_year,article_month,article_day,slug_keywords,slug_length,is_live_blog,topic_country
0,australia-news/live/2025/oct/08/australia-poli...,liveblog,Australia News,2025-10-08 02:58:31+00:00,Australia politics live: Price says leaks risk...,Australia politics live: Price says leaks risk...,Follow live,Krishani Dhanji,7812,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,live,2025,oct,8,"[australia, politics, live, gaza, rally, parli...",15,True,australia
1,us-news/live/2025/oct/07/donald-trump-national...,liveblog,Us News,2025-10-08 02:06:57+00:00,Oregon’s governor says she told Kristi Noem ‘t...,Oregon’s governor says she told Kristi Noem ‘t...,"Tina Kotek, Oregon’s governor, says in stateme...","Robert Mackey (Now); Shrai Popat, Lucy Campbel...",10078,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,live,2025,oct,7,"[donald, trump, national, guard, pam, bondi, m...",15,True,us
2,music/2025/oct/08/dolly-parton-health-problems...,article,Music,2025-10-08 01:21:12+00:00,"Dolly Parton ‘going to be just fine’, singer’s...","Dolly Parton ‘going to be just fine’, singer’s...",Freida Parton says she has been ‘up all night ...,Sian Cain,426,theguardian.com,...,raw/2025-10-08/articles_20251008_025937.json,transform_guardian_v2,None,2025,oct,8,"[dolly, parton, health, problems, sister, pray...",11,False,None


In [15]:
print(df_p.columns.tolist())


['article_id', 'type', 'sectionName', 'webPublicationDate', 'webTitle', 'fields_headline', 'fields_trailText', 'fields_byline', 'fields_wordcount', 'fields_publication', 'fields_thumbnail', 'fields_body', 'pillarName', 'webUrl', 'title_length', 'headline_length', 'days_since_publication', 'has_thumbnail', 'ingested_at', 'source_system', 'raw_s3_key', 'processed_by', 'article_type', 'article_year', 'article_month', 'article_day', 'slug_keywords', 'slug_length', 'is_live_blog', 'topic_country']
